test_001

In [3]:
"""
VerimGören — Noktadan (Google Maps linki) ürün uygunluk skoru üretimi (v0.2)

Kullanım:
1) Aşağıda LINK değişkenine Google Haritalar bağlantısını yapıştırın. 
   - Örnek biçimler:
     - https://www.google.com/maps/place/41°52'10.0"N+33°53'29.0"E
     - https://www.google.com/maps/@41.869395,12.494279,12z
     - Sadece "lat,lon" şeklinde de verebilirsiniz: "41.8694,12.4943"
2) Dosyayı çalıştırın.
3) Çıktı: Konuma ait özet iklim değerleri, (opsiyonel) rakım/ışık örneklemesi ve
   pilot ürünler için 0–100 arası uygunluk skorları + açıklamalar.

Notlar:
- İklim verileri NASA POWER Climatology (1991–2020 aylık ortalamalar) API üzerinden çekilir.
- ET0 hesapları aylık Hargreaves (FAO-56) ile yapılır (ışınım/gün uzunluğu enlemden türetilir).
- Eşikler/parametreler bölgeye göre kalibre edilmelidir; bu sürüm yalın bir başlangıçtır.
- (Opsiyonel) yerel rasterlardan rakım/ışık örneklemesi için ELEV_CROPPED_PATH ve 
  LIGHT_CROPPED_PATH dosyalarını sağlarsanız rapora eklenir.
"""

import re
import json
import math
import calendar
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import requests

try:
    import rasterio  # opsiyonel
except Exception:  # rasterio yoksa sessiz devam edelim
    rasterio = None

# -----------------------------
# 0) Kullanıcı girişi
# -----------------------------

LINK = "https://www.google.com/maps/place/41°52'10.0\"N+33°53'29.0\"E"

# Opsiyonel raster yolları (varsa rapora eklenir)
ELEV_CROPPED_PATH = "../data/processed/srtm_turkiye_cropped.tif"
LIGHT_CROPPED_PATH = "../data/processed/viirs_light_2024_turkey.tif"

# -----------------------------
# 1) Yardımcılar — Koordinat, DMS/Decimal ayrıştırma
# -----------------------------

def decimal_to_dms(degree: float) -> Tuple[int, int, float]:
    is_negative = degree < 0
    degree = abs(degree)
    d = int(degree)
    m_float = (degree - d) * 60
    m = int(m_float)
    s = (m_float - m) * 60
    return (-d if is_negative else d, m, s)


def dms_to_decimal(deg: float, minute: float, sec: float, hemi: str) -> float:
    sign = -1 if hemi.upper() in ["S", "W"] else 1
    return sign * (abs(deg) + minute / 60.0 + sec / 3600.0)


DMS_PATTERN = re.compile(
    r"(?P<deg>\d{1,3})°(?P<min>\d{1,2})'(?P<sec>[\d\.]+)\"(?P<hemi>[NSEW])"
)


def parse_google_maps_link(text: str) -> Tuple[float, float]:
    """Google Maps linkinden veya 'lat,lon' metninden (lat, lon) döndürür."""
    text = text.strip()

    # 1) '@lat,lon' biçimi
    if "@" in text:
        try:
            after = text.split("@", 1)[1]
            nums = re.split(r"[^-\d\.]+", after)
            nums = [n for n in nums if n not in ("", None)]
            lat = float(nums[0])
            lon = float(nums[1])
            return lat, lon
        except Exception:
            pass

    # 2) DMS kalıbı (iki adet beklenir: lat ve lon)
    dms_hits = DMS_PATTERN.findall(text)
    if len(dms_hits) >= 2:
        lat_d, lat_m, lat_s, lat_h = dms_hits[0]
        lon_d, lon_m, lon_s, lon_h = dms_hits[1]
        lat = dms_to_decimal(float(lat_d), float(lat_m), float(lat_s), lat_h)
        lon = dms_to_decimal(float(lon_d), float(lon_m), float(lon_s), lon_h)
        return lat, lon

    # 3) 'lat,lon' düz metin
    if "," in text:
        a, b = text.split(",", 1)
        return float(a), float(b)

    raise ValueError("Koordinat ayrıştırılamadı. Lütfen bir Google Maps linki veya 'lat,lon' verin.")


# -----------------------------
# 2) NASA POWER — aylık klimatoloji çekimi
# -----------------------------

POWER_PARAMS = [
    "T2M",         # 2m Temperature (°C)
    "T2M_MAX",     # 2m Tmax (°C)
    "T2M_MIN",     # 2m Tmin (°C)
    "PRECTOTCORR", # Precipitation (mm/day)
    "ALLSKY_SFC_SW_DWN", # All-sky shortwave (kWh/m2/day)
    "RH2M",        # 2m Relative Humidity (%)
    "WS2M",        # 2m Wind Speed (m/s)
]

MONTHS = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
MONTH_IDX = {m:i+1 for i,m in enumerate(MONTHS)}


def fetch_power_climatology(lat: float, lon: float) -> Dict[str, List[float]]:
    params = ",".join(POWER_PARAMS)
    url = (
        "https://power.larc.nasa.gov/api/temporal/climatology/point"
        f"?parameters={params}&community=AG&longitude={lon}&latitude={lat}&format=JSON"
    )
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    p = data["properties"]["parameter"]
    out = {}
    for key in POWER_PARAMS:
        series = [p[key][m] for m in MONTHS]
        out[key] = series
    return out


# -----------------------------
# 3) Aylık ET0 (Hargreaves) ve yardımcı iklim hesapları
# -----------------------------

GSC = 0.0820  # MJ m-2 min-1
PI = math.pi


def day_of_year_for_month_center(month: int) -> int:
    # Ay ortasını yaklaşıkla (15. gün). Şubat için 15 kabul edilir.
    return [15, 46, 75, 105, 135, 162, 198, 228, 258, 288, 318, 344][month-1]


def extraterrestrial_radiation_MJm2day(lat_deg: float, month: int) -> float:
    """FAO-56 formülleri ile ay ortası için Ra (MJ/m2/day)."""
    phi = math.radians(lat_deg)
    J = day_of_year_for_month_center(month)
    dr = 1 + 0.033 * math.cos(2*PI*J/365)
    delta = 0.409 * math.sin(2*PI*J/365 - 1.39)
    ws = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24*60/PI) * GSC * dr * (ws*math.sin(phi)*math.sin(delta) + math.cos(phi)*math.cos(delta)*math.sin(ws))
    return Ra  # MJ/m2/day


def eto_hargreaves_monthly(Tmean: float, Tmax: float, Tmin: float, Ra_MJm2day: float) -> float:
    # FAO-56 Hargreaves: ETo (mm/day) = 0.0023 * (Tmean + 17.8) * sqrt(Tmax - Tmin) * Ra
    dT = max(Tmax - Tmin, 0.1)  # çok küçük aralıkları stabilize et
    return 0.0023 * (Tmean + 17.8) * math.sqrt(dT) * Ra_MJm2day


def days_in_month(m: int) -> int:
    # Klimatoloji için tipik gün sayıları
    return [31,28,31,30,31,30,31,31,30,31,30,31][m-1]


# -----------------------------
# 4) Ürün profilleri ve uygunluk skoru
# -----------------------------

@dataclass
class CropProfile:
    name: str
    tbase: float
    season_months: List[int]
    gdd_target: int
    gdd_upper: int
    kc_mean: float
    alt_min: float = 0.0
    alt_max: float = 2000.0
    frost_tolerance: float = 0.0  # °C; sezon Tmin altına inerse ceza (örn. -2 tolerans)
    heat_thresh: float = 35.0     # °C; sezon Tmax bunun üstüne çıkarsa ceza


CROPS: List[CropProfile] = [
    CropProfile("Buğday (kış)", tbase=5,  season_months=[10,11,12,1,2,3,4,5,6], gdd_target=1200, gdd_upper=2000, kc_mean=0.95, alt_max=1800, frost_tolerance=-10, heat_thresh=34),
    CropProfile("Mısır (dane)", tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2400, kc_mean=1.15, alt_max=1400, frost_tolerance=0,   heat_thresh=36),
    CropProfile("Ayçiçeği",     tbase=6,  season_months=[4,5,6,7,8,9],         gdd_target=1200, gdd_upper=2000, kc_mean=1.00, alt_max=1500, frost_tolerance=0,   heat_thresh=35),
    CropProfile("Domates",      tbase=10, season_months=[5,6,7,8,9],           gdd_target=1400, gdd_upper=2200, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Biber",        tbase=10, season_months=[5,6,7,8,9],           gdd_target=1300, gdd_upper=2100, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Bağ (üzüm)",   tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2500, kc_mean=0.90, alt_max=1300, frost_tolerance=-2,  heat_thresh=38),
    CropProfile("Zeytin",       tbase=12, season_months=[3,4,5,6,7,8,9,10,11], gdd_target=1800, gdd_upper=3000, kc_mean=0.70, alt_max=1000, frost_tolerance=-5,  heat_thresh=40),
]


def trapezoid_score(x: float, a: float, b: float, c: float, d: float) -> float:
    """[a..b] artış, [b..c] 1, [c..d] azalış; dışında 0. x, a<=b<=c<=d."""
    if x <= a or x >= d:
        return 0.0
    if b <= x <= c:
        return 1.0
    if a < x < b:
        return (x - a) / (b - a)
    # c < x < d
    return (d - x) / (d - c)


def suitability_for_crop(crop: CropProfile, monthly: Dict[str, List[float]], lat: float, alt: Optional[float]) -> Tuple[float, Dict[str, float]]:
    # 1) GDD (aylık ort Tmean → gün sayısıyla çarp)
    Tmean = monthly["T2M"]
    Tmax = monthly["T2M_MAX"]
    Tmin = monthly["T2M_MIN"]
    Pmm_day = monthly["PRECTOTCORR"]  # mm/day

    gdd = 0.0
    et0_sum = 0.0
    p_sum = 0.0

    for m in crop.season_months:
        idx = m - 1
        d = days_in_month(m)
        # GDD: max(0, Tmean - tbase) * days
        gdd += max(Tmean[idx] - crop.tbase, 0.0) * d
        # ET0: Hargreaves
        Ra = extraterrestrial_radiation_MJm2day(lat, m)
        et0 = eto_hargreaves_monthly(Tmean[idx], Tmax[idx], Tmin[idx], Ra)
        et0_sum += et0 * d
        p_sum += max(Pmm_day[idx], 0.0) * d

    # 2) Su dengesi: P / (ET0 * Kc)
    etc_sum = et0_sum * crop.kc_mean
    ratio = p_sum / max(etc_sum, 1e-6)
    # hedef ~1; 0.8–1.2 aralığı 1.0 skor, uçlarda azalsın
    water_score = trapezoid_score(ratio, 0.5, 0.8, 1.2, 1.8)

    # 3) GDD skoru: target civarı iyi; çok düşük/çok yüksek düşsün
    gdd_score = trapezoid_score(gdd, 0.6*crop.gdd_target, 0.9*crop.gdd_target, 1.2*crop.gdd_target, crop.gdd_upper)

    # 4) Isı stresi (sezon içi ayların Tmax maks değeri)
    season_tmax = max(Tmax[m-1] for m in crop.season_months)
    if season_tmax <= crop.heat_thresh:
        heat_score = 1.0
    else:
        # 35→0.9, 40→0.2, 45→0; lineer penalizasyon
        heat_score = max(0.0, 1.0 - (season_tmax - crop.heat_thresh) / 10.0)

    # 5) Don riski (sezon içi Tmin min)
    season_tmin = min(Tmin[m-1] for m in crop.season_months)
    if season_tmin >= crop.frost_tolerance:
        frost_score = 1.0
    else:
        # her 5°C fazla don için güçlü ceza
        frost_score = max(0.0, 1.0 - (crop.frost_tolerance - season_tmin) / 5.0)

    # 6) Rakım kısıtı (opsiyonel)
    if alt is None:
        alt_score = 1.0
    else:
        if alt < crop.alt_min - 100 or alt > crop.alt_max + 200:
            alt_score = 0.0
        else:
            # [min, min+100] yükselen; [max-200, max+200] düşen plato
            alt_score = min(
                1.0 if alt >= crop.alt_min + 100 else max(0.0, (alt - (crop.alt_min - 100)) / 200.0),
                1.0 if alt <= crop.alt_max - 200 else max(0.0, ((crop.alt_max + 200) - alt) / 400.0)
            )

    # Bileşik skor
    base = 0.45 * gdd_score + 0.35 * water_score + 0.20 * heat_score
    final = base * frost_score * alt_score

    detail = {
        "gdd": gdd,
        "gdd_score": gdd_score,
        "p_sum_mm": p_sum,
        "et0_sum_mm": et0_sum,
        "etc_sum_mm": etc_sum,
        "water_ratio": ratio,
        "water_score": water_score,
        "season_tmax": season_tmax,
        "heat_score": heat_score,
        "season_tmin": season_tmin,
        "frost_score": frost_score,
        "altitude_m": alt if alt is not None else float("nan"),
        "alt_score": alt_score,
    }
    return final * 100.0, detail


# -----------------------------
# 5) Opsiyonel raster örnekleme (rakım/ışık)
# -----------------------------

def sample_raster_value(path: str, lon: float, lat: float) -> Optional[float]:
    if rasterio is None:
        return None
    try:
        with rasterio.open(path) as ds:
            row, col = ds.index(lon, lat)
            arr = ds.read(1)
            val = float(arr[row, col])
            if ds.nodata is not None and val == ds.nodata:
                return None
            return val
    except Exception:
        return None


# -----------------------------
# 6) Raporlama
# -----------------------------

def summarize_monthly(m: Dict[str, List[float]]) -> str:
    def fmt(x):
        return f"{x:5.1f}"
    lines = [
        "Ay   Tmn   Tmx   Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)",
        "---  ----  ----  ----- -----------  -------------------",
    ]
    for i, mon in enumerate(MONTHS, start=1):
        line = f"{mon} {fmt(m['T2M_MIN'][i-1])} {fmt(m['T2M_MAX'][i-1])} {fmt(m['T2M'][i-1])}     {m['PRECTOTCORR'][i-1]:6.2f}              {m['ALLSKY_SFC_SW_DWN'][i-1]:5.2f}"
        lines.append(line)
    return "\n".join(lines)


# -----------------------------
# 7) Ana akış
# -----------------------------

def main():
    lat, lon = parse_google_maps_link(LINK)
    print(f"Konum: {lat:.5f}, {lon:.5f}")

    # İklim (aylık klimatoloji)
    monthly = fetch_power_climatology(lat, lon)

    # (opsiyonel) raster örneklemeleri
    elev = sample_raster_value(ELEV_CROPPED_PATH, lon, lat)
    light = sample_raster_value(LIGHT_CROPPED_PATH, lon, lat)

    # Özet tablo
    print("\n— Aylık İklim Özeti (NASA POWER, 1991–2020 klimatoloji) —")
    print(summarize_monthly(monthly))

    if elev is not None:
        print(f"\nRakım (SRTM): {elev:.0f} m")
    if light is not None:
        print(f"Gece ışığı (VIIRS): {light:.3f}")

    # Ürün uygunlukları
    results = []
    for crop in CROPS:
        score, detail = suitability_for_crop(crop, monthly, lat, elev)
        results.append((crop.name, score, detail))

    results.sort(key=lambda x: x[1], reverse=True)

    print("\n— Ürün Uygunluk Skoru (0–100) —")
    for name, score, det in results:
        reason = (
            f"GDD≈{det['gdd']:.0f} (skor {det['gdd_score']:.2f}); "
            f"Su indeksi P/ETc≈{det['water_ratio']:.2f} (skor {det['water_score']:.2f}); "
            f"Tmax_sezon≈{det['season_tmax']:.1f} (ısı skor {det['heat_score']:.2f}); "
            f"Tmin_sezon≈{det['season_tmin']:.1f} (don skor {det['frost_score']:.2f})"
        )
        if not math.isnan(det['altitude_m']):
            reason += f"; Rakım≈{det['altitude_m']:.0f} m (skor {det['alt_score']:.2f})"
        print(f"• {name:12s} → {score:5.1f} | {reason}")

    top3 = ", ".join([r[0] for r in results[:3]])
    print(f"\nÖnerilen ilk 3: {top3}")


if __name__ == "__main__":
    main()


Konum: 41.86944, 33.89139

— Aylık İklim Özeti (NASA POWER, 1991–2020 klimatoloji) —
Ay   Tmn   Tmx   Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)
---  ----  ----  ----- -----------  -------------------
JAN  -6.1  17.4   4.5       2.83               6.12
FEB  -7.7  20.7   5.0       2.34               8.96
MAR  -5.1  23.3   7.0       2.12              12.82
APR  -2.0  26.1  10.3       1.51              17.41
MAY   4.7  31.7  15.3       1.60              20.78
JUN   9.5  35.1  19.7       1.70              23.92
JUL  14.4  33.6  22.2       0.94              25.02
AUG  15.4  37.2  22.7       0.83              22.45
SEP  10.6  34.2  19.6       1.76              16.66
OCT   5.2  31.2  15.3       2.23              11.16
NOV   0.4  26.6  10.9       2.14               7.68
DEC  -5.5  22.9   6.7       3.38               5.37

Rakım (SRTM): 1503 m
Gece ışığı (VIIRS): 0.000

— Ürün Uygunluk Skoru (0–100) —
• Buğday (kış) →  55.5 | GDD≈1531 (skor 0.84); Su indeksi P/ETc≈0.26 (skor 0.00); Tmax_sezon≈35.1 

##__
test_002

In [4]:
"""
VerimGören — Noktadan (lat,lon) → Google Maps linki → ürün uygunluk skoru (v0.3)

Kullanım akışı:
1) Program çalışınca "lat,lon" biçiminde koordinat isteyecek (örn: 38.946838, 28.080573).
2) Girilen koordinattan Google Maps linkini üretecek ve ekrana yazacak.
3) İklim verilerini (NASA POWER 1991–2020 aylık ort.), (opsiyonel) raster örnekleri
   ve pilot ürünler için uygunluk skorlarını üretecek.

Notlar:
- İklim verileri NASA POWER Climatology API (AG community) üzerinden çekilir.
- ET0 hesapları aylık Hargreaves (FAO-56) ile yapılır (ışınım/gün uzunluğu enlemden türetilir).
- (Opsiyonel) rasterlar sağlanırsa rapora rakım/ışık eklenir.
"""

import re
import json
import math
import calendar
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import requests

try:
    import rasterio  # opsiyonel
except Exception:  # rasterio yoksa sessiz devam edelim
    rasterio = None

# -----------------------------
# 0) Kullanıcı girişi
# -----------------------------
# Opsiyonel raster yolları (varsa rapora eklenir)
ELEV_CROPPED_PATH = "../data/processed/srtm_turkiye_cropped.tif"
LIGHT_CROPPED_PATH = "../data/processed/viirs_light_2024_turkey.tif"

# -----------------------------
# 1) Yardımcılar — Koordinat, DMS/Decimal ayrıştırma & link üretimi
# -----------------------------

def decimal_to_dms(degree: float) -> Tuple[int, int, float]:
    is_negative = degree < 0
    degree = abs(degree)
    d = int(degree)
    m_float = (degree - d) * 60
    m = int(m_float)
    s = (m_float - m) * 60
    return (-d if is_negative else d, m, s)

def dms_to_decimal(deg: float, minute: float, sec: float, hemi: str) -> float:
    sign = -1 if hemi.upper() in ["S", "W"] else 1
    return sign * (abs(deg) + minute / 60.0 + sec / 3600.0)

DMS_PATTERN = re.compile(
    r"(?P<deg>\d{1,3})°(?P<min>\d{1,2})'(?P<sec>[\d\.]+)\"(?P<hemi>[NSEW])"
)

def parse_latlon_text(text: str) -> Tuple[float, float]:
    """
    'lat,lon' düz metninden (lat, lon) döndürür.
    Boşlukları ve nokta/virgül ayracını tolere eder.
    """
    text = text.strip()
    # ';' veya birden fazla boşluk vs. için normalize et
    parts = re.split(r"[,\s;]+", text)
    parts = [p for p in parts if p]
    if len(parts) < 2:
        raise ValueError("Lütfen 'lat,lon' biçiminde iki sayı girin. Örn: 38.946838, 28.080573")
    lat = float(parts[0])
    lon = float(parts[1])
    if not (-90.0 <= lat <= 90.0 and -180.0 <= lon <= 180.0):
        raise ValueError("Geçersiz aralık: enlem [-90,90], boylam [-180,180] olmalı.")
    return lat, lon

def google_maps_url_from_latlon(lat: float, lon: float, zoom: int = 12) -> str:
    """
    Koordinattan pratik Google Maps URL üretir.
    - @lat,lon,zoomz biçimi paylaşım için uygundur
    """
    return f"https://www.google.com/maps/@{lat:.6f},{lon:.6f},{zoom}z"

def google_maps_place_url(lat: float, lon: float) -> str:
    """
    Alternatif place sorgusu:
    """
    return f"https://www.google.com/maps?q={lat:.6f},{lon:.6f}"

# -----------------------------
# 2) NASA POWER — aylık klimatoloji çekimi
# -----------------------------

POWER_PARAMS = [
    "T2M",         # 2m Temperature (°C)
    "T2M_MAX",     # 2m Tmax (°C)
    "T2M_MIN",     # 2m Tmin (°C)
    "PRECTOTCORR", # Precipitation (mm/day)
    "ALLSKY_SFC_SW_DWN", # All-sky shortwave (kWh/m2/day)
    "RH2M",        # 2m Relative Humidity (%)
    "WS2M",        # 2m Wind Speed (m/s)
]

MONTHS = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]
MONTH_IDX = {m:i+1 for i,m in enumerate(MONTHS)}

def fetch_power_climatology(lat: float, lon: float) -> Dict[str, List[float]]:
    params = ",".join(POWER_PARAMS)
    url = (
        "https://power.larc.nasa.gov/api/temporal/climatology/point"
        f"?parameters={params}&community=AG&longitude={lon}&latitude={lat}&format=JSON"
    )
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()
    p = data["properties"]["parameter"]
    out = {}
    for key in POWER_PARAMS:
        series = [p[key][m] for m in MONTHS]
        out[key] = series
    return out

# -----------------------------
# 3) Aylık ET0 (Hargreaves) ve yardımcı iklim hesapları
# -----------------------------

GSC = 0.0820  # MJ m-2 min-1
PI = math.pi

def day_of_year_for_month_center(month: int) -> int:
    # Ay ortasını yaklaşıkla (15. gün). Şubat için 15 kabul edilir.
    return [15, 46, 75, 105, 135, 162, 198, 228, 258, 288, 318, 344][month-1]

def extraterrestrial_radiation_MJm2day(lat_deg: float, month: int) -> float:
    """FAO-56 formülleri ile ay ortası için Ra (MJ/m2/day)."""
    phi = math.radians(lat_deg)
    J = day_of_year_for_month_center(month)
    dr = 1 + 0.033 * math.cos(2*PI*J/365)
    delta = 0.409 * math.sin(2*PI*J/365 - 1.39)
    ws = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24*60/PI) * GSC * dr * (ws*math.sin(phi)*math.sin(delta) + math.cos(phi)*math.cos(delta)*math.sin(ws))
    return Ra  # MJ/m2/day

def eto_hargreaves_monthly(Tmean: float, Tmax: float, Tmin: float, Ra_MJm2day: float) -> float:
    # FAO-56 Hargreaves: ETo (mm/day) = 0.0023 * (Tmean + 17.8) * sqrt(Tmax - Tmin) * Ra
    dT = max(Tmax - Tmin, 0.1)  # çok küçük aralıkları stabilize et
    return 0.0023 * (Tmean + 17.8) * math.sqrt(dT) * Ra_MJm2day

def days_in_month(m: int) -> int:
    # Klimatoloji için tipik gün sayıları
    return [31,28,31,30,31,30,31,31,30,31,30,31][m-1]

# -----------------------------
# 4) Ürün profilleri ve uygunluk skoru
# -----------------------------

@dataclass
class CropProfile:
    name: str
    tbase: float
    season_months: List[int]
    gdd_target: int
    gdd_upper: int
    kc_mean: float
    alt_min: float = 0.0
    alt_max: float = 2000.0
    frost_tolerance: float = 0.0  # °C; sezon Tmin altına inerse ceza (örn. -2 tolerans)
    heat_thresh: float = 35.0     # °C; sezon Tmax bunun üstüne çıkarsa ceza

CROPS: List[CropProfile] = [
    CropProfile("Buğday (kış)", tbase=5,  season_months=[10,11,12,1,2,3,4,5,6], gdd_target=1200, gdd_upper=2000, kc_mean=0.95, alt_max=1800, frost_tolerance=-10, heat_thresh=34),
    CropProfile("Mısır (dane)", tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2400, kc_mean=1.15, alt_max=1400, frost_tolerance=0,   heat_thresh=36),
    CropProfile("Ayçiçeği",     tbase=6,  season_months=[4,5,6,7,8,9],         gdd_target=1200, gdd_upper=2000, kc_mean=1.00, alt_max=1500, frost_tolerance=0,   heat_thresh=35),
    CropProfile("Domates",      tbase=10, season_months=[5,6,7,8,9],           gdd_target=1400, gdd_upper=2200, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Biber",        tbase=10, season_months=[5,6,7,8,9],           gdd_target=1300, gdd_upper=2100, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Bağ (üzüm)",   tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2500, kc_mean=0.90, alt_max=1300, frost_tolerance=-2,  heat_thresh=38),
    CropProfile("Zeytin",       tbase=12, season_months=[3,4,5,6,7,8,9,10,11], gdd_target=1800, gdd_upper=3000, kc_mean=0.70, alt_max=1000, frost_tolerance=-5,  heat_thresh=40),
]

def trapezoid_score(x: float, a: float, b: float, c: float, d: float) -> float:
    """[a..b] artış, [b..c] 1, [c..d] azalış; dışında 0. x, a<=b<=c<=d."""
    if x <= a or x >= d:
        return 0.0
    if b <= x <= c:
        return 1.0
    if a < x < b:
        return (x - a) / (b - a)
    # c < x < d
    return (d - x) / (d - c)

def suitability_for_crop(crop: CropProfile, monthly: Dict[str, List[float]], lat: float, alt: Optional[float]) -> Tuple[float, Dict[str, float]]:
    # 1) GDD (aylık ort Tmean → gün sayısıyla çarp)
    Tmean = monthly["T2M"]
    Tmax = monthly["T2M_MAX"]
    Tmin = monthly["T2M_MIN"]
    Pmm_day = monthly["PRECTOTCORR"]  # mm/day

    gdd = 0.0
    et0_sum = 0.0
    p_sum = 0.0

    for m in crop.season_months:
        idx = m - 1
        d = days_in_month(m)
        # GDD: max(0, Tmean - tbase) * days
        gdd += max(Tmean[idx] - crop.tbase, 0.0) * d
        # ET0: Hargreaves
        Ra = extraterrestrial_radiation_MJm2day(lat, m)
        et0 = eto_hargreaves_monthly(Tmean[idx], Tmax[idx], Tmin[idx], Ra)
        et0_sum += et0 * d
        p_sum += max(Pmm_day[idx], 0.0) * d

    # 2) Su dengesi: P / (ET0 * Kc)
    etc_sum = et0_sum * crop.kc_mean
    ratio = p_sum / max(etc_sum, 1e-6)
    # hedef ~1; 0.8–1.2 aralığı 1.0 skor, uçlarda azalsın
    water_score = trapezoid_score(ratio, 0.5, 0.8, 1.2, 1.8)

    # 3) GDD skoru: target civarı iyi; çok düşük/çok yüksek düşsün
    gdd_score = trapezoid_score(gdd, 0.6*crop.gdd_target, 0.9*crop.gdd_target, 1.2*crop.gdd_target, crop.gdd_upper)

    # 4) Isı stresi (sezon içi ayların Tmax maks değeri)
    season_tmax = max(Tmax[m-1] for m in crop.season_months)
    if season_tmax <= crop.heat_thresh:
        heat_score = 1.0
    else:
        # 35→0.9, 40→0.2, 45→0; lineer penalizasyon
        heat_score = max(0.0, 1.0 - (season_tmax - crop.heat_thresh) / 10.0)

    # 5) Don riski (sezon içi Tmin min)
    season_tmin = min(Tmin[m-1] for m in crop.season_months)
    if season_tmin >= crop.frost_tolerance:
        frost_score = 1.0
    else:
        # her 5°C fazla don için güçlü ceza
        frost_score = max(0.0, 1.0 - (crop.frost_tolerance - season_tmin) / 5.0)

    # 6) Rakım kısıtı (opsiyonel)
    if alt is None:
        alt_score = 1.0
    else:
        if alt < crop.alt_min - 100 or alt > crop.alt_max + 200:
            alt_score = 0.0
        else:
            # [min, min+100] yükselen; [max-200, max+200] düşen plato
            alt_score = min(
                1.0 if alt >= crop.alt_min + 100 else max(0.0, (alt - (crop.alt_min - 100)) / 200.0),
                1.0 if alt <= crop.alt_max - 200 else max(0.0, ((crop.alt_max + 200) - alt) / 400.0)
            )

    # Bileşik skor
    base = 0.45 * gdd_score + 0.35 * water_score + 0.20 * heat_score
    final = base * frost_score * alt_score

    detail = {
        "gdd": gdd,
        "gdd_score": gdd_score,
        "p_sum_mm": p_sum,
        "et0_sum_mm": et0_sum,
        "etc_sum_mm": etc_sum,
        "water_ratio": ratio,
        "water_score": water_score,
        "season_tmax": season_tmax,
        "heat_score": heat_score,
        "season_tmin": season_tmin,
        "frost_score": frost_score,
        "altitude_m": alt if alt is not None else float("nan"),
        "alt_score": alt_score,
    }
    return final * 100.0, detail

# -----------------------------
# 5) Opsiyonel raster örnekleme (rakım/ışık)
# -----------------------------

def sample_raster_value(path: str, lon: float, lat: float) -> Optional[float]:
    if rasterio is None:
        return None
    try:
        with rasterio.open(path) as ds:
            row, col = ds.index(lon, lat)
            arr = ds.read(1)
            val = float(arr[row, col])
            if ds.nodata is not None and val == ds.nodata:
                return None
            return val
    except Exception:
        return None

# -----------------------------
# 6) Raporlama
# -----------------------------

MONTHS_TR = ["OCA","ŞUB","MAR","NİS","MAY","HAZ","TEM","AĞU","EYL","EKİ","KAS","ARA"]

def summarize_monthly(m: Dict[str, List[float]]) -> str:
    def fmt(x):
        return f"{x:5.1f}"
    lines = [
        "Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)",
        "---  ----  ----  ----- -----------  -------------------",
    ]
    for i, mon in enumerate(MONTHS_TR, start=1):
        line = f"{mon} {fmt(m['T2M_MIN'][i-1])} {fmt(m['T2M_MAX'][i-1])} {fmt(m['T2M'][i-1])}     {m['PRECTOTCORR'][i-1]:6.2f}              {m['ALLSKY_SFC_SW_DWN'][i-1]:5.2f}"
        lines.append(line)
    return "\n".join(lines)

# -----------------------------
# 7) Ana akış
# -----------------------------

def main():
    # 1) Kullanıcıdan lat,lon iste
    raw = input("Lütfen 'lat,lon' girin (örn: 38.946838, 28.080573): ").strip()
    lat, lon = parse_latlon_text(raw)

    # 2) Google Maps linkini üret & yazdır
    maps_url = google_maps_url_from_latlon(lat, lon, zoom=12)
    maps_place_url = google_maps_place_url(lat, lon)
    print(f"\nKonum: {lat:.6f}, {lon:.6f}")
    print(f"Google Maps (kamera): {maps_url}")
    print(f"Google Maps (sorgu) : {maps_place_url}")

    # 3) İklim (aylık klimatoloji)
    print("\nVeriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...")
    monthly = fetch_power_climatology(lat, lon)

    # 4) (opsiyonel) raster örneklemeleri
    elev = sample_raster_value(ELEV_CROPPED_PATH, lon, lat)
    light = sample_raster_value(LIGHT_CROPPED_PATH, lon, lat)

    # 5) Özet tablo
    print("\n— Aylık İklim Özeti —")
    print(summarize_monthly(monthly))

    if elev is not None:
        print(f"\nRakım (SRTM): {elev:.0f} m")
    if light is not None:
        print(f"Gece ışığı (VIIRS): {light:.3f}")

    # 6) Ürün uygunlukları
    results = []
    for crop in CROPS:
        score, detail = suitability_for_crop(crop, monthly, lat, elev)
        results.append((crop.name, score, detail))

    results.sort(key=lambda x: x[1], reverse=True)

    print("\n— Ürün Uygunluk Skoru (0–100) —")
    for name, score, det in results:
        reason = (
            f"GDD≈{det['gdd']:.0f} (skor {det['gdd_score']:.2f}); "
            f"Su indeksi P/ETc≈{det['water_ratio']:.2f} (skor {det['water_score']:.2f}); "
            f"Tmax_sezon≈{det['season_tmax']:.1f} (ısı skor {det['heat_score']:.2f}); "
            f"Tmin_sezon≈{det['season_tmin']:.1f} (don skor {det['frost_score']:.2f})"
        )
        if not math.isnan(det['altitude_m']):
            reason += f"; Rakım≈{det['altitude_m']:.0f} m (skor {det['alt_score']:.2f})"
        print(f"• {name:12s} → {score:5.1f} | {reason}")

    top3 = ", ".join([r[0] for r in results[:3]])
    print(f"\nÖnerilen ilk 3: {top3}")

if __name__ == "__main__":
    main()


Lütfen 'lat,lon' girin (örn: 38.946838, 28.080573):  38.946838, 28.080573



Konum: 38.946838, 28.080573
Google Maps (kamera): https://www.google.com/maps/@38.946838,28.080573,12z
Google Maps (sorgu) : https://www.google.com/maps?q=38.946838,28.080573

Veriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...

— Aylık İklim Özeti —
Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)
---  ----  ----  ----- -----------  -------------------
OCA -13.3  16.8   3.2       3.25               7.66
ŞUB -11.6  20.6   4.7       2.65              10.18
MAR  -6.7  26.7   7.7       1.83              14.93
NİS  -4.5  28.9  11.8       1.65              19.61
MAY  -0.3  37.3  17.1       1.27              23.81
HAZ   8.4  41.6  21.9       0.91              27.12
TEM  11.9  42.3  25.7       0.26              28.52
AĞU  11.6  41.1  26.2       0.19              25.34
EYL   5.9  40.2  21.4       0.77              20.02
EKİ  -1.6  36.0  15.4       1.65              14.09
KAS  -5.8  28.6   9.6       2.17               9.57
ARA  -8.5  20.9   4.8       2.70               7.01

Rakım (

test_003

In [5]:
"""
VerimGören — Noktadan (Google Maps linki **veya** lat,lon) → ürün uygunluk skoru (v0.4)

Kullanım akışı:
1) Program çalışınca konumu **Google Maps linki** YA DA **"lat,lon"** biçiminde girin.
   Örnekler:  
   - https://www.google.com/maps/@38.946838,28.080573,12z  
   - https://www.google.com/maps/place/41°52'10.0\"N+33°53'29.0\"E  
   - 38.946838, 28.080573
2) Betik Google Maps linkini yeniden yazdırır (kamera & sorgu linki), NASA POWER 1991–2020 aylık
   klimatolojiyi çeker (cache'li & retry'li), (opsiyonel) rasterlardan rakım/ışık örnekler,
   pilot ürünler için 0–100 uygunluk skorlarını ve gerekçelerini üretir.
3) Çıktıları ekrana yazar, ayrıca **results.json** ve **results.csv** dosyalarına kaydeder.

İyileştirmeler (v0.4):
- Giriş: Link **veya** lat,lon (akıllı ayrıştırma)
- Sulama modu anahtarı: rainfed / irrigated → su bileşeni ağırlığı dinamik
- Ürün profilleri **harici JSON** ile yüklenebilir (yoksa dahili varsayılanlar)
- NASA POWER çağrıları: retry + basit disk cache (./cache/power_climatology_cache.json)
- JSON/CSV çıktı dökümleri
"""

import os
import re
import csv
import json
import math
from pathlib import Path
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional

import time
import requests

try:
    import rasterio  # opsiyonel
except Exception:  # rasterio yoksa sessiz devam
    rasterio = None

# -----------------------------
# 0) Kullanıcı değişkenleri
# -----------------------------

# Sulama modu: "rainfed" (kuru koşul) veya "irrigated" (sulamalı)
IRRIGATION_MODE = "rainfed"  # "rainfed" | "irrigated"

# Ürün profili harici konfigürasyon dosyası (opsiyonel)
CROP_CONFIG_PATH = Path("./crop_profiles.json")

# Opsiyonel raster yolları (varsa rapora eklenir)
ELEV_CROPPED_PATH = Path("../data/processed/srtm_turkiye_cropped.tif")
LIGHT_CROPPED_PATH = Path("../data/processed/viirs_light_2024_turkey.tif")

# Cache dosyası (NASA POWER klimatoloji)
CACHE_DIR = Path("./cache")
CACHE_DIR.mkdir(parents=True, exist_ok=True)
POWER_CACHE_FILE = CACHE_DIR / "power_climatology_cache.json"

# -----------------------------
# 1) Yardımcılar — Koordinat ayrıştırma & link üretimi
# -----------------------------

DMS_PATTERN = re.compile(r"(?P<deg>\d{1,3})°(?P<min>\d{1,2})'(?P<sec>[\d\.]+)\"(?P<hemi>[NSEW])")


def dms_to_decimal(deg: float, minute: float, sec: float, hemi: str) -> float:
    sign = -1 if hemi.upper() in ["S", "W"] else 1
    return sign * (abs(deg) + minute / 60.0 + sec / 3600.0)


def parse_latlon_text(text: str) -> Tuple[float, float]:
    """'lat,lon' düz metninden (lat, lon) döndürür; boşluk/virgül/; tolere eder."""
    text = text.strip()
    parts = re.split(r"[,\s;]+", text)
    parts = [p for p in parts if p]
    if len(parts) < 2:
        raise ValueError("Lütfen 'lat,lon' biçiminde iki sayı girin. Örn: 38.946838, 28.080573")
    lat = float(parts[0])
    lon = float(parts[1])
    if not (-90.0 <= lat <= 90.0 and -180.0 <= lon <= 180.0):
        raise ValueError("Geçersiz aralık: enlem [-90,90], boylam [-180,180] olmalı.")
    return lat, lon


def parse_google_maps_link(text: str) -> Optional[Tuple[float, float]]:
    """Google Maps linkinden (lat, lon) döndürür; '@lat,lon' ve DMS biçimlerini destekler."""
    text = text.strip()
    # 1) '@lat,lon' biçimi
    if "@" in text:
        try:
            after = text.split("@", 1)[1]
            nums = re.split(r"[^-\d\.]+", after)
            nums = [n for n in nums if n]
            lat = float(nums[0])
            lon = float(nums[1])
            return lat, lon
        except Exception:
            pass
    # 2) DMS kalıbı (iki adet: lat, lon)
    dms_hits = DMS_PATTERN.findall(text)
    if len(dms_hits) >= 2:
        (ld, lm, ls, lh), (od, om, os, oh) = dms_hits[0], dms_hits[1]
        lat = dms_to_decimal(float(ld), float(lm), float(ls), lh)
        lon = dms_to_decimal(float(od), float(om), float(os), oh)
        return lat, lon
    return None


def parse_any_location(text: str) -> Tuple[float, float]:
    link = parse_google_maps_link(text)
    if link is not None:
        return link
    return parse_latlon_text(text)


def google_maps_url_from_latlon(lat: float, lon: float, zoom: int = 12) -> str:
    return f"https://www.google.com/maps/@{lat:.6f},{lon:.6f},{zoom}z"


def google_maps_place_url(lat: float, lon: float) -> str:
    return f"https://www.google.com/maps?q={lat:.6f},{lon:.6f}"

# -----------------------------
# 2) NASA POWER — aylık klimatoloji (retry + cache)
# -----------------------------

POWER_PARAMS = [
    "T2M",         # 2m Temperature (°C)
    "T2M_MAX",     # 2m Tmax (°C)
    "T2M_MIN",     # 2m Tmin (°C)
    "PRECTOTCORR", # Precipitation (mm/day)
    "ALLSKY_SFC_SW_DWN", # All-sky shortwave (kWh/m2/day)
    "RH2M",        # 2m Relative Humidity (%)
    "WS2M",        # 2m Wind Speed (m/s)
]

MONTHS = ["JAN","FEB","MAR","APR","MAY","JUN","JUL","AUG","SEP","OCT","NOV","DEC"]


def _load_power_cache() -> Dict:
    if POWER_CACHE_FILE.exists():
        try:
            return json.loads(POWER_CACHE_FILE.read_text(encoding="utf-8"))
        except Exception:
            return {}
    return {}


def _save_power_cache(cache: Dict):
    try:
        POWER_CACHE_FILE.write_text(json.dumps(cache), encoding="utf-8")
    except Exception:
        pass


def _power_cache_key(lat: float, lon: float) -> str:
    # 0.01° (~1 km) çözünürlükte yuvarla, anahtar kısa kalsın
    return f"{round(lat, 2):.2f},{round(lon, 2):.2f}"


def fetch_power_climatology(lat: float, lon: float, retries: int = 3, backoff: float = 1.5) -> Dict[str, List[float]]:
    cache = _load_power_cache()
    key = _power_cache_key(lat, lon)
    if key in cache:
        return cache[key]

    params = ",".join(POWER_PARAMS)
    url = (
        "https://power.larc.nasa.gov/api/temporal/climatology/point"
        f"?parameters={params}&community=AG&longitude={lon}&latitude={lat}&format=JSON"
    )
    last_err = None
    for i in range(retries):
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            data = r.json()
            p = data["properties"]["parameter"]
            out = {}
            for keyp in POWER_PARAMS:
                out[keyp] = [p[keyp][m] for m in MONTHS]
            cache[key] = out
            _save_power_cache(cache)
            return out
        except Exception as e:
            last_err = e
            if i < retries - 1:
                time.sleep(backoff ** (i+1))
    # son denemede de hata:
    raise RuntimeError(f"NASA POWER isteği başarısız: {last_err}")

# -----------------------------
# 3) Aylık ET0 (Hargreaves) ve yardımcı iklim hesapları
# -----------------------------

GSC = 0.0820  # MJ m-2 min-1
PI = math.pi


def day_of_year_for_month_center(month: int) -> int:
    return [15, 46, 75, 105, 135, 162, 198, 228, 258, 288, 318, 344][month-1]


def extraterrestrial_radiation_MJm2day(lat_deg: float, month: int) -> float:
    phi = math.radians(lat_deg)
    J = day_of_year_for_month_center(month)
    dr = 1 + 0.033 * math.cos(2*PI*J/365)
    delta = 0.409 * math.sin(2*PI*J/365 - 1.39)
    ws = math.acos(-math.tan(phi) * math.tan(delta))
    Ra = (24*60/PI) * GSC * dr * (ws*math.sin(phi)*math.sin(delta) + math.cos(phi)*math.cos(delta)*math.sin(ws))
    return Ra  # MJ/m2/day


def eto_hargreaves_monthly(Tmean: float, Tmax: float, Tmin: float, Ra_MJm2day: float) -> float:
    dT = max(Tmax - Tmin, 0.1)
    return 0.0023 * (Tmean + 17.8) * math.sqrt(dT) * Ra_MJm2day


def days_in_month(m: int) -> int:
    return [31,28,31,30,31,30,31,31,30,31,30,31][m-1]

# -----------------------------
# 4) Ürün profilleri ve uygunluk
# -----------------------------

@dataclass
class CropProfile:
    name: str
    tbase: float
    season_months: List[int]
    gdd_target: int
    gdd_upper: int
    kc_mean: float
    alt_min: float = 0.0
    alt_max: float = 2000.0
    frost_tolerance: float = 0.0  # °C
    heat_thresh: float = 35.0     # °C


# Dahili varsayılan profiller (harici JSON yoksa kullanılır)
DEFAULT_CROPS: List[CropProfile] = [
    CropProfile("Buğday (kış)", tbase=5,  season_months=[10,11,12,1,2,3,4,5,6], gdd_target=1200, gdd_upper=2000, kc_mean=0.95, alt_max=1800, frost_tolerance=-10, heat_thresh=34),
    CropProfile("Mısır (dane)", tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2400, kc_mean=1.15, alt_max=1400, frost_tolerance=0,   heat_thresh=36),
    CropProfile("Ayçiçeği",     tbase=6,  season_months=[4,5,6,7,8,9],         gdd_target=1200, gdd_upper=2000, kc_mean=1.00, alt_max=1500, frost_tolerance=0,   heat_thresh=35),
    CropProfile("Domates",      tbase=10, season_months=[5,6,7,8,9],           gdd_target=1400, gdd_upper=2200, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Biber",        tbase=10, season_months=[5,6,7,8,9],           gdd_target=1300, gdd_upper=2100, kc_mean=1.10, alt_max=1200, frost_tolerance=2,   heat_thresh=36),
    CropProfile("Bağ (üzüm)",   tbase=10, season_months=[4,5,6,7,8,9],         gdd_target=1600, gdd_upper=2500, kc_mean=0.90, alt_max=1300, frost_tolerance=-2,  heat_thresh=38),
    CropProfile("Zeytin",       tbase=12, season_months=[3,4,5,6,7,8,9,10,11], gdd_target=1800, gdd_upper=3000, kc_mean=0.70, alt_max=1000, frost_tolerance=-5,  heat_thresh=40),
]


def load_crop_profiles(path: Path) -> List[CropProfile]:
    if not path.exists():
        return DEFAULT_CROPS
    try:
        data = json.loads(path.read_text(encoding="utf-8"))
        profiles: List[CropProfile] = []
        for item in data:
            profiles.append(CropProfile(
                name=item["name"],
                tbase=float(item["tbase"]),
                season_months=[int(x) for x in item["season_months"]],
                gdd_target=int(item["gdd_target"]),
                gdd_upper=int(item.get("gdd_upper", int(item["gdd_target"]) * 2)),
                kc_mean=float(item.get("kc_mean", 1.0)),
                alt_min=float(item.get("alt_min", 0.0)),
                alt_max=float(item.get("alt_max", 2000.0)),
                frost_tolerance=float(item.get("frost_tolerance", 0.0)),
                heat_thresh=float(item.get("heat_thresh", 35.0)),
            ))
        return profiles
    except Exception as e:
        print(f"Uyarı: {path} okunamadı ({e}); dahili varsayılanlar kullanılacak.")
        return DEFAULT_CROPS


def trapezoid_score(x: float, a: float, b: float, c: float, d: float) -> float:
    if x <= a or x >= d:
        return 0.0
    if b <= x <= c:
        return 1.0
    if a < x < b:
        return (x - a) / (b - a)
    return (d - x) / (d - c)


def suitability_for_crop(crop: CropProfile, monthly: Dict[str, List[float]], lat: float, alt: Optional[float], irrigation_mode: str = IRRIGATION_MODE) -> Tuple[float, Dict[str, float]]:
    Tmean = monthly["T2M"]
    Tmax = monthly["T2M_MAX"]
    Tmin = monthly["T2M_MIN"]
    Pmm_day = monthly["PRECTOTCORR"]  # mm/day

    gdd = 0.0
    et0_sum = 0.0
    p_sum = 0.0

    for m in crop.season_months:
        idx = m - 1
        d = days_in_month(m)
        gdd += max(Tmean[idx] - crop.tbase, 0.0) * d
        Ra = extraterrestrial_radiation_MJm2day(lat, m)
        et0 = eto_hargreaves_monthly(Tmean[idx], Tmax[idx], Tmin[idx], Ra)
        et0_sum += et0 * d
        p_sum += max(Pmm_day[idx], 0.0) * d

    etc_sum = et0_sum * crop.kc_mean
    ratio = p_sum / max(etc_sum, 1e-6)
    water_score = trapezoid_score(ratio, 0.5, 0.8, 1.2, 1.8)

    gdd_score = trapezoid_score(gdd, 0.6*crop.gdd_target, 0.9*crop.gdd_target, 1.2*crop.gdd_target, crop.gdd_upper)

    season_tmax = max(Tmax[m-1] for m in crop.season_months)
    if season_tmax <= crop.heat_thresh:
        heat_score = 1.0
    else:
        heat_score = max(0.0, 1.0 - (season_tmax - crop.heat_thresh) / 10.0)

    season_tmin = min(Tmin[m-1] for m in crop.season_months)
    if season_tmin >= crop.frost_tolerance:
        frost_score = 1.0
    else:
        frost_score = max(0.0, 1.0 - (crop.frost_tolerance - season_tmin) / 5.0)

    # Rakım
    if alt is None:
        alt_score = 1.0
    else:
        if alt < crop.alt_min - 100 or alt > crop.alt_max + 200:
            alt_score = 0.0
        else:
            alt_score = min(
                1.0 if alt >= crop.alt_min + 100 else max(0.0, (alt - (crop.alt_min - 100)) / 200.0),
                1.0 if alt <= crop.alt_max - 200 else max(0.0, ((crop.alt_max + 200) - alt) / 400.0)
            )

    # Ağırlıklar — sulama moduna göre
    water_w = 0.35 if irrigation_mode == "rainfed" else 0.15
    base = 0.45 * gdd_score + water_w * water_score + 0.20 * heat_score
    final = base * frost_score * alt_score

    detail = {
        "gdd": gdd,
        "gdd_score": gdd_score,
        "p_sum_mm": p_sum,
        "et0_sum_mm": et0_sum,
        "etc_sum_mm": etc_sum,
        "water_ratio": ratio,
        "water_score": water_score,
        "season_tmax": season_tmax,
        "heat_score": heat_score,
        "season_tmin": season_tmin,
        "frost_score": frost_score,
        "altitude_m": alt if alt is not None else float("nan"),
        "alt_score": alt_score,
    }
    return final * 100.0, detail

# -----------------------------
# 5) Opsiyonel raster örnekleme
# -----------------------------

def sample_raster_value(path: Path, lon: float, lat: float) -> Optional[float]:
    if rasterio is None or not path.exists():
        return None
    try:
        with rasterio.open(path) as ds:
            row, col = ds.index(lon, lat)
            arr = ds.read(1)
            val = float(arr[row, col])
            if ds.nodata is not None and val == ds.nodata:
                return None
            return val
    except Exception:
        return None

# -----------------------------
# 6) Raporlama & çıktı dökümleri
# -----------------------------

MONTHS_TR = ["OCA","ŞUB","MAR","NİS","MAY","HAZ","TEM","AĞU","EYL","EKİ","KAS","ARA"]


def summarize_monthly(m: Dict[str, List[float]]) -> str:
    def fmt(x):
        return f"{x:5.1f}"
    lines = [
        "Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)",
        "---  ----  ----  ----- -----------  -------------------",
    ]
    for i, mon in enumerate(MONTHS_TR, start=1):
        line = f"{mon} {fmt(m['T2M_MIN'][i-1])} {fmt(m['T2M_MAX'][i-1])} {fmt(m['T2M'][i-1])}     {m['PRECTOTCORR'][i-1]:6.2f}              {m['ALLSKY_SFC_SW_DWN'][i-1]:5.2f}"
        lines.append(line)
    return "\n".join(lines)


def write_results_json(path: Path, payload: Dict):
    try:
        path.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")
    except Exception as e:
        print(f"Uyarı: JSON yazılamadı: {e}")


def write_results_csv(path: Path, rows: List[Dict[str, object]]):
    try:
        if not rows:
            return
        cols = list(rows[0].keys())
        with path.open("w", newline="", encoding="utf-8") as f:
            w = csv.DictWriter(f, fieldnames=cols)
            w.writeheader()
            for r in rows:
                w.writerow(r)
    except Exception as e:
        print(f"Uyarı: CSV yazılamadı: {e}")

# -----------------------------
# 7) Ana akış
# -----------------------------

def main():
    raw = input("Konum girin (Google Maps linki YA DA 'lat,lon'): ").strip()
    lat, lon = parse_any_location(raw)

    maps_url = google_maps_url_from_latlon(lat, lon, zoom=12)
    maps_place_url = google_maps_place_url(lat, lon)

    print(f"\nKonum: {lat:.6f}, {lon:.6f}")
    print(f"Google Maps (kamera): {maps_url}")
    print(f"Google Maps (sorgu) : {maps_place_url}")

    print("\nVeriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...")
    monthly = fetch_power_climatology(lat, lon)

    elev = sample_raster_value(ELEV_CROPPED_PATH, lon, lat)
    light = sample_raster_value(LIGHT_CROPPED_PATH, lon, lat)

    print("\n— Aylık İklim Özeti —")
    print(summarize_monthly(monthly))

    if elev is not None:
        print(f"\nRakım (SRTM): {elev:.0f} m")
    if light is not None:
        print(f"Gece ışığı (VIIRS): {light:.3f}")

    crops = load_crop_profiles(CROP_CONFIG_PATH)
    results = []
    rows_csv: List[Dict[str, object]] = []

    for crop in crops:
        score, det = suitability_for_crop(crop, monthly, lat, elev, irrigation_mode=IRRIGATION_MODE)
        results.append((crop.name, score, det))
        rows_csv.append({
            "crop": crop.name,
            "score": round(score, 1),
            "gdd": round(det["gdd"], 0),
            "gdd_score": round(det["gdd_score"], 2),
            "water_ratio": round(det["water_ratio"], 2),
            "water_score": round(det["water_score"], 2),
            "season_tmax": round(det["season_tmax"], 1),
            "heat_score": round(det["heat_score"], 2),
            "season_tmin": round(det["season_tmin"], 1),
            "frost_score": round(det["frost_score"], 2),
            "altitude_m": None if math.isnan(det["altitude_m"]) else int(det["altitude_m"]),
            "alt_score": round(det["alt_score"], 2),
        })

    results.sort(key=lambda x: x[1], reverse=True)

    print("\n— Ürün Uygunluk Skoru (0–100) —")
    for name, score, det in results:
        reason = (
            f"GDD≈{det['gdd']:.0f} (skor {det['gdd_score']:.2f}); "
            f"Su indeksi P/ETc≈{det['water_ratio']:.2f} (skor {det['water_score']:.2f}); "
            f"Tmax_sezon≈{det['season_tmax']:.1f} (ısı skor {det['heat_score']:.2f}); "
            f"Tmin_sezon≈{det['season_tmin']:.1f} (don skor {det['frost_score']:.2f})"
        )
        if not math.isnan(det['altitude_m']):
            reason += f"; Rakım≈{det['altitude_m']:.0f} m (skor {det['alt_score']:.2f})"
        print(f"• {name:12s} → {score:5.1f} | {reason}")

    top3 = ", ".join([r[0] for r in results[:3]])
    print(f"\nSulama modu: {IRRIGATION_MODE}")
    print(f"Önerilen ilk 3: {top3}")

    # JSON & CSV dökümleri
    payload = {
        "location": {"lat": lat, "lon": lon, "maps_camera": maps_url, "maps_query": maps_place_url},
        "irrigation_mode": IRRIGATION_MODE,
        "monthly": monthly,
        "elevation_m": None if elev is None else float(elev),
        "night_light": None if light is None else float(light),
        "results_sorted": [
            {"crop": name, "score": round(score,1), "detail": det} for name, score, det in results
        ],
    }
    write_results_json(Path("results.json"), payload)
    write_results_csv(Path("results.csv"), rows_csv)


if __name__ == "__main__":
    main()


Konum girin (Google Maps linki YA DA 'lat,lon'):  39.401066669083654, 35.85977717718739



Konum: 39.401067, 35.859777
Google Maps (kamera): https://www.google.com/maps/@39.401067,35.859777,12z
Google Maps (sorgu) : https://www.google.com/maps?q=39.401067,35.859777

Veriler çekiliyor (NASA POWER, 1991–2020 aylık ort.)...

— Aylık İklim Özeti —
Ay   Tmin  Tmax  Tavg  Yağış(mm/g)  Radyasyon(kWh/m²/g)
---  ----  ----  ----- -----------  -------------------
OCA -22.1  12.2  -2.8       1.63               7.36
ŞUB -23.2  16.9  -0.8       1.33              10.97
MAR -16.3  22.9   3.1       1.62              14.42
NİS  -8.7  26.6   7.6       1.46              19.08
MAY  -3.1  30.6  12.6       1.82              22.89
HAZ   2.1  34.0  17.1       1.25              26.56
TEM   4.9  38.5  20.3       0.40              27.66
AĞU   5.1  38.5  20.9       0.24              25.29
EYL  -0.5  35.6  16.8       0.59              19.88
EKİ  -8.0  28.3  11.1       0.95              13.49
KAS -16.9  21.6   4.1       1.29               9.09
ARA -24.2  17.7  -0.8       1.42               6.53

Rakım (